In [ ]:
import pandas as pd
#inital data cleaning
df = pd.read_csv('file.csv')
#format mobile number to 10 digits
df['Mobile No.'] = df['Mobile No.'].astype(str).str.replace(' ', '')

df1 = df[df['Mobile No.'].str.contains(',')]
df1['Mobile No.'] = df1['Mobile No.'].str.split(',')
df1['Coordinator'] = df1['Coordinator'].apply(lambda x: x.split(',') if len(x.split(',')) != 1 else [x for i in range(2)])
df1 = df1.explode(['Mobile No.', 'Coordinator'])

# remove all elements with multiple mobile numbers
df = df[~df['Mobile No.'].str.contains(',')]

# add the new rows to the original dataframe
df = df.append(df1, ignore_index=True)
df

In [ ]:
def make_vcard(
        first_name,
        last_name,
        phone):
    return [
        'BEGIN:VCARD',
        'VERSION:3.0',
        f'N:{last_name};{first_name}',
        f'FN:{first_name} {last_name}',
        f'TEL;TYPE=CELL:+91{phone}',
        'END:VCARD'
    ]

In [ ]:
make_vcard("first_name", "Last_name", "1234567890")

In [ ]:
vcards = []

for index, row in df.iterrows():
    # print(index, row['Mobile No.'], row['Coordinator'], row['Clubs'])
    # print(row['Mobile No.'], row['Coordinator'], row['Clubs'])
    # split into first and last name
    names = row['Coordinator'].split(' ')
    if len(names) == 1:
        first_name = names[0]
        last_name = ''
    else:
        first_name = names[0]
        last_name = names[1]

    vcard = make_vcard(first_name, f"{last_name} {row['Clubs']}", row['Mobile No.'])
    vcards.append(vcard)

vcards

In [ ]:
with open('contacts.vcf', 'w') as f:
    for vcard in vcards:
        f.writelines([l + '\n' for l in vcard])